In [15]:
import requests
import json
import pandas as pd
import numpy as np

graph_id = "QmX9Ucyfb6EMJC5ZeB61h1SuZ5HFKwJymA6We2xyzaQ1Tg"
url = f"https://api.thegraph.com/subgraphs/id/{graph_id}"


PASS = "PASS"
FAIL = "FAIL"
MATCH_PERCENT_TOLERANCE = 0.00001 # Much be within 0.001% of eachother
PAGE_LENGTH = 1000

check_summary = {}


In [16]:
# Helpers

def list_items_match(a: list, b: list) -> list:
    exact_match = (a == b)
    within_tolerance = abs(1 - a / b) < MATCH_PERCENT_TOLERANCE
    return exact_match | within_tolerance 
    
def perform_request(query, variables = {}):
    res = requests.post(url, json={"query": query, "variables": variables})
    return json.loads(res.text)["data"]

def perform_pagenation_request(pageable_query, key, varibales={}):
    all_found = False
    skip = 0
    data = []

    while(not all_found):
        query_variables = varibales
        query_variables["page_length"] = PAGE_LENGTH
        query_variables["skip"] = skip

        res = requests.post(url, json={"query": pageable_query, "variables": query_variables}) 

        new_data = json.loads(res.text)["data"][key]
        data += new_data

        all_found = (len(new_data) != PAGE_LENGTH)
        skip += PAGE_LENGTH
    
    return {key: data}




In [ ]:
# Check total accounts equals protocol account
accounts_query = """query($skip: Int!, $page_length: Int!) {
    accounts(first: $page_length, skip: $skip) {
        id
    }
}"""

protocol_query = """query {
    protocols {
        cumulativeUniqueUsers
    }
}"""

account_data = perform_pagenation_request(accounts_query, "accounts")
protocol_data = perform_request(protocol_query)
num_accounts = len(account_data["accounts"])
cumulative_users = protocol_data["protocols"][0]["cumulativeUniqueUsers"]

result = PASS if num_accounts == cumulative_users else FAIL 

print(f"num_accounts: {num_accounts}, cumulative_users: {cumulative_users}, result: {result}")

check_summary["unique_user_check"] = result


In [ ]:
# Check cumulative for protocol match sum for pools

query = """query {
    lendingProtocols {
        totalValueLockedUSD
        cumulativeSupplySideRevenueUSD
        cumulativeProtocolSideRevenueUSD
        cumulativeTotalRevenueUSD
        totalDepositBalanceUSD
        cumulativeDepositUSD
        totalBorrowBalanceUSD
        cumulativeBorrowUSD
        cumulativeLiquidateUSD
    }
    markets {
        totalValueLockedUSD
        cumulativeSupplySideRevenueUSD
        cumulativeProtocolSideRevenueUSD
        cumulativeTotalRevenueUSD
        totalDepositBalanceUSD
        cumulativeDepositUSD
        totalBorrowBalanceUSD
        cumulativeBorrowUSD
        cumulativeLiquidateUSD
    }
}"""

data = perform_request(query)
protcol_data = data["lendingProtocols"][0]
market_data = data["markets"]
df = pd.DataFrame(market_data)
df = df.apply(pd.to_numeric, errors='coerce') 
market_sums = df.sum(axis=0)

compare_df = pd.DataFrame({"market_sums": market_sums, "protocol_cumulatives": protcol_data})
compare_df = compare_df.apply(pd.to_numeric, errors='coerce') 
compare_df["result"] = np.where(list_items_match(compare_df["market_sums"], compare_df["protocol_cumulatives"]), PASS, FAIL)
print(compare_df)

result = PASS if (compare_df["result"].to_numpy() == PASS).all() else FAIL

check_summary["protocol_cumulatives_match_check"] = result



                                   market_sums  protocol_cumulatives result
totalValueLockedUSD               7.150305e+08          7.150305e+08   PASS
cumulativeSupplySideRevenueUSD    3.911867e+07          3.911867e+07   PASS
cumulativeProtocolSideRevenueUSD  9.794812e+07          9.794812e+07   PASS
cumulativeTotalRevenueUSD         1.370668e+08          1.370668e+08   PASS
totalDepositBalanceUSD            7.026939e+08          7.026939e+08   PASS
cumulativeDepositUSD              1.186261e+09          1.186261e+09   PASS
totalBorrowBalanceUSD             6.270124e+08          6.270124e+08   PASS
cumulativeBorrowUSD               1.452832e+09          1.452832e+09   PASS
cumulativeLiquidateUSD            0.000000e+00          0.000000e+00   PASS


In [ ]:
# Check financial snapshots add up to protocol_cumulatives

snapshot_query = """query($skip: Int!, $page_length: Int!) {
    financialsDailySnapshots(first: $page_length, skip: $skip) {
        dailySupplySideRevenueUSD
        dailyProtocolSideRevenueUSD
        dailyDepositUSD
        dailyBorrowUSD
        dailyLiquidateUSD
    }
}"""

protocol_query = """query {
    lendingProtocols {
        cumulativeSupplySideRevenueUSD
        cumulativeProtocolSideRevenueUSD
        cumulativeDepositUSD
        cumulativeBorrowUSD
        cumulativeLiquidateUSD
    }
}"""


protocol_data = perform_request(protocol_query)["lendingProtocols"][0]
snapshot_data = perform_pagenation_request(snapshot_query, "financialsDailySnapshots")["financialsDailySnapshots"]
df = pd.DataFrame(snapshot_data)
df = df.apply(pd.to_numeric, errors='coerce') 
df.rename(columns = {'dailySupplySideRevenueUSD':'cumulativeSupplySideRevenueUSD', 'dailyProtocolSideRevenueUSD':'cumulativeProtocolSideRevenueUSD', "dailyDepositUSD": "cumulativeDepositUSD", "dailyBorrowUSD": "cumulativeBorrowUSD", "dailyLiquidateUSD": "cumulativeLiquidateUSD"}, inplace = True)
sums = df.sum(axis=0)

compare_df = pd.DataFrame({"snapshot_sums": sums, "protocol_cumulatives": protocol_data})
compare_df = compare_df.apply(pd.to_numeric, errors='coerce') 
compare_df["result"] = np.where(list_items_match(compare_df["snapshot_sums"], compare_df["protocol_cumulatives"]), PASS, FAIL)
print(compare_df)

result = PASS if (compare_df["result"].to_numpy() == PASS).all() else FAIL

check_summary["financial_snapshots_check"] = result


                                  snapshot_sums  protocol_cumulatives result
cumulativeSupplySideRevenueUSD     3.911867e+07          3.911867e+07   PASS
cumulativeProtocolSideRevenueUSD   9.794812e+07          9.794812e+07   PASS
cumulativeDepositUSD               1.186261e+09          1.186261e+09   PASS
cumulativeBorrowUSD                1.452832e+09          1.452832e+09   PASS
cumulativeLiquidateUSD             0.000000e+00          0.000000e+00   PASS


In [ ]:
# Check market snapshots add up to market cumulatives

market_query = """query {
    markets(first: 1) {
        id
        cumulativeDepositUSD
        cumulativeBorrowUSD
        cumulativeLiquidateUSD
    }
}"""

hourly_snapshot_query = """query($skip: Int!, $page_length: Int!, $market: String!) {
    marketHourlySnapshots(first: $page_length, skip: $skip, where: {market: $market}) {
        hourlyDepositUSD
        hourlyBorrowUSD
        hourlyLiquidateUSD
    }
}"""

daily_snapshot_query = """query($skip: Int!, $page_length: Int!, $market: String!) {
    marketDailySnapshots(first: $page_length, skip: $skip, where: {market: $market}) {
        dailyDepositUSD
        dailyBorrowUSD
        dailyLiquidateUSD
    }
}"""

data = perform_request(query) 
market_data = perform_request(market_query)["markets"][0]
market_id = market_data["id"]
filtered_market_data = {"cumulativeDepositUSD": float(market_data["cumulativeDepositUSD"]), "cumulativeBorrowUSD": float(market_data["cumulativeBorrowUSD"]), "cumulativeLiquidateUSD": float(market_data["cumulativeLiquidateUSD"])}

query_variables = {"market": market_id}
hourly_snapshot_data = perform_pagenation_request(hourly_snapshot_query, "marketHourlySnapshots", query_variables)["marketHourlySnapshots"]
hourly_snapshot_df = pd.DataFrame(hourly_snapshot_data)
hourly_snapshot_df = hourly_snapshot_df.apply(pd.to_numeric, errors='coerce') 
hourly_snapshot_df.rename(columns = {'hourlyDepositUSD':'cumulativeDepositUSD', 'hourlyBorrowUSD':'cumulativeBorrowUSD', "hourlyLiquidateUSD": "cumulativeLiquidateUSD"}, inplace = True)
hourly_snapshot_sums = hourly_snapshot_df.sum(axis=0) 


daily_snapshot_data = perform_pagenation_request(daily_snapshot_query, "marketDailySnapshots", query_variables)["marketDailySnapshots"]
daily_snapshot_df = pd.DataFrame(daily_snapshot_data)
daily_snapshot_df = daily_snapshot_df.apply(pd.to_numeric, errors='coerce') 
daily_snapshot_df.rename(columns = {'dailyDepositUSD':'cumulativeDepositUSD', 'dailyBorrowUSD':'cumulativeBorrowUSD', "dailyLiquidateUSD": "cumulativeLiquidateUSD"}, inplace = True)
daily_snapshot_sums = daily_snapshot_df.sum(axis=0) 

compare_df = pd.DataFrame({"market_data": filtered_market_data, "hourly_snapshot_sums": hourly_snapshot_sums, "daily_snapshot_sums": daily_snapshot_sums})
compare_df["result"] = np.where(list_items_match(compare_df["market_data"], compare_df["hourly_snapshot_sums"]) & list_items_match(compare_df["market_data"], compare_df["daily_snapshot_sums"]), PASS, FAIL)
print(compare_df)

result = PASS if (compare_df["result"].to_numpy() == PASS).all() else FAIL
check_summary["market_snapshots_check"] = result



                         market_data  hourly_snapshot_sums  \
cumulativeDepositUSD    5.010916e+07          5.010916e+07   
cumulativeBorrowUSD     4.801522e+07          4.801522e+07   
cumulativeLiquidateUSD  0.000000e+00          0.000000e+00   

                        daily_snapshot_sums result  
cumulativeDepositUSD           5.010916e+07   PASS  
cumulativeBorrowUSD            4.801522e+07   PASS  
cumulativeLiquidateUSD         0.000000e+00   PASS  


In [ ]:
# Summarize results

print(check_summary)

{'unique_user_check': 'PASS', 'protocol_cumulatives_match_check': 'PASS', 'financial_snapshots_check': 'PASS', 'market_snapshots_check': 'PASS'}


In [ ]:
market_query = """query {
    markets{
       inputToken {
        name
        }
        _cumulativeInterest
        _cumulativePoolDelegateRevenue
        _cumulativeTreasuryRevenue
		_cumulativeDeposit
        _cumulativeBorrow
        _cumulativeLiquidate
    }
}"""

data = perform_request(query) 
market_data = perform_request(market_query)["markets"]

usdc_market_data = [entry for entry in market_data if entry["inputToken"]["name"] == "USD Coin"]
eth_market_data = [entry for entry in market_data if entry["inputToken"]["name"] == "Wrapped Ether"]

usdc_market_df = pd.DataFrame(usdc_market_data)
usdc_market_df = usdc_market_df.apply(pd.to_numeric, errors='coerce') 

eth_market_df = pd.DataFrame(eth_market_data)
eth_market_df = eth_market_df.apply(pd.to_numeric, errors='coerce') 
eth_market_df

usdc_market_sums = usdc_market_df.sum(axis=0) 
eth_market_sums = eth_market_df.sum(axis=0) 
print(eth_market_df)
print(eth_market_sums)

result = pd.concat([usdc_market_sums, eth_market_sums])


# print(result)
result.to_csv("exports/result.csv")



   inputToken  _cumulativeInterest  _cumulativePoolDelegateRevenue  \
0         NaN         1.290542e+20            16131780821917805622   
1         NaN         1.151390e+20                               0   

   _cumulativeTreasuryRevenue  _cumulativeDeposit  _cumulativeBorrow  \
0                1.222971e+21        1.932443e+22       1.860900e+22   
1                9.723191e+20        1.480188e+22       1.479500e+22   

   _cumulativeLiquidate  
0                     0  
1                     0  
inputToken                        0.000000e+00
_cumulativeInterest               2.441932e+20
_cumulativePoolDelegateRevenue   -2.314963e+18
_cumulativeTreasuryRevenue        2.195290e+21
_cumulativeDeposit                3.412631e+22
_cumulativeBorrow                 3.340400e+22
_cumulativeLiquidate              0.000000e+00
dtype: float64
